In [1]:
import tensorflow as tf
import pickle
import numpy as np
import timeit, time
import load_cifar

In [2]:
filePath = '/home/lkumari/data/cifar-10-batches-py/'

<h1>Hyperparameter</h1>

In [3]:
EPOCHS = 10
BATCH_SIZE = 128
lr = 0.01
num_classes = 10
num_batches = 5

In [4]:
tf.reset_default_graph()

<h1>Placeholder</h1>

In [5]:
X = tf.placeholder(tf.float32, [None, 32, 32, 3])
Y = tf.placeholder(tf.int32, [None, 10])


mean = 0.0
stddev = 0.1

def initialize_weight(shape, mean, stddev):
    W = tf.truncated_normal(shape=shape, mean = mean, stddev = stddev)
    return tf.Variable(W)



weights = {
    'W_conv1': tf.Variable(initialize_weight([5,5,3,6], mean, stddev),name='W_conv1'),
    'W_conv2': tf.Variable(initialize_weight([5,5,6,16], mean, stddev),name='W_conv2'),
    'W_fc1': tf.Variable(initialize_weight([5*5*16,120], mean, stddev),name='W_fc1'),
    'W_fc2': tf.Variable(initialize_weight([120,84], mean, stddev),name='W_fc2'),
    'W_out': tf.Variable(initialize_weight([84,num_classes], mean, stddev),name='W_out')
}

biases = {
    'b_conv1': tf.Variable(tf.zeros(shape=[6]),name='b_conv1'),
    'b_conv2': tf.Variable(tf.zeros(shape=[16]),name='b_conv2'),
    'b_fc1': tf.Variable(tf.zeros(shape=[120]),name='b_fc1'),
    'b_fc2': tf.Variable(tf.zeros(shape=[84]),name='b_fc2'),
    'b_out': tf.Variable(tf.zeros(shape=[num_classes]),name='b_out')
}
    
def batch_norm(input):
    return tf.contrib.layers.batch_norm(input, decay=0.9, epsilon=1e-3, center=True, scale=True)

Instructions for updating:
Colocations handled automatically by placer.


<h1>LeNet-5</h1>

In [6]:

def LeNet(x):
    conv1 = tf.nn.conv2d(x, weights['W_conv1'], strides=[1,1,1,1], padding='VALID') + biases['b_conv1']
    conv1 = tf.nn.relu(batch_norm(conv1))
    conv1 = tf.nn.max_pool(conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')
    
    conv2 = tf.nn.conv2d(conv1, weights['W_conv2'], strides=[1,1,1,1], padding='VALID') + biases['b_conv2']
    conv2 = tf.nn.relu(batch_norm(conv2))
    conv2 = tf.nn.max_pool(conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')
    
    fc_input = tf.contrib.layers.flatten(conv2)
    
    fc1 = tf.matmul(fc_input, weights['W_fc1']) + biases['b_fc1']
    fc1 = tf.nn.relu(batch_norm(fc1))
    
    fc2 = tf.matmul(fc1, weights['W_fc2']) + biases['b_fc2']
    fc2 = tf.nn.relu(batch_norm(fc2))
    
    out = tf.matmul(fc2, weights['W_out']) + biases['b_out']
    return out

<h1>Cost and Optimization</h1>

In [7]:
#predicted labels
logits = LeNet(X)

#define loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=Y),name='loss')
#define optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=lr)
train_op = optimizer.minimize(loss)

#compare the predicted labels with true labels
correct_pred = tf.equal(tf.argmax(logits,1),tf.argmax(Y,1))

#compute the accuracy by taking average
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32),name='accuracy')

#Initialize the variables
init = tf.global_variables_initializer()


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.flatten instead.


<h1>Training, validation and testing</h1>
<h2>Train your model only 10 epochs.</h2>
<h2>1.Print out validation accuracy after each training epoch</h2>
<h2>2.Print out training time for each training epoch</h2>
<h2>3.Print out testing accuracy</h2>

In [170]:
val_accuracy = []
test_accuracy = []
time_taken = []

with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    sess.run(init)
    
    for ep in range(EPOCHS):
        start_time = time.time()
        for batch_num in range(1, num_batches+1):
            for batch_features, batch_labels in load_cifar.load_preprocessed_training_batch(batch_num, BATCH_SIZE):
                bat_features = load_cifar.features_reshape(batch_features)
                train_loss, acc, _ = sess.run(fetches=[loss, accuracy, train_op], feed_dict={X: bat_features, Y: batch_labels})
        end_time = time.time()
        time_s = end_time - start_time
        
        val_features, val_labels = load_cifar.load_preprocessed_validation_batch()
        val_features = load_cifar.features_reshape(val_features)
        val_acc = sess.run(fetches=[accuracy], feed_dict={X:val_features, Y:val_labels})

        test_features, test_labels = load_cifar.load_preprocessed_test()
        test_features = load_cifar.features_reshape(test_features)
        test_acc = sess.run(fetches=[accuracy], feed_dict={X:test_features, Y:test_labels})
        
        val_accuracy.append(val_acc[0])
        test_accuracy.append(test_acc[0])
        time_taken.append(time_s)
                
        print("Epoch {}: training loss = {}, training_time = {}, val_accuracy = {}, test_accuracy = {}".format(
        ep, train_loss, time_s, val_acc[0], test_acc[0]))

                

Epoch 0: training loss = 1.578251838684082, training_time = 4.613215684890747, val_accuracy = 0.5217999815940857, test_accuracy = 0.5181999802589417
Epoch 1: training loss = 1.2737643718719482, training_time = 4.29104208946228, val_accuracy = 0.5483999848365784, test_accuracy = 0.5483999848365784
Epoch 2: training loss = 1.0994131565093994, training_time = 4.266815423965454, val_accuracy = 0.5633999705314636, test_accuracy = 0.5670999884605408
Epoch 3: training loss = 1.0075470209121704, training_time = 4.218254566192627, val_accuracy = 0.579200029373169, test_accuracy = 0.5782999992370605
Epoch 4: training loss = 0.8623148798942566, training_time = 4.209357261657715, val_accuracy = 0.5785999894142151, test_accuracy = 0.5795000195503235
Epoch 5: training loss = 0.6508873701095581, training_time = 4.241961479187012, val_accuracy = 0.6019999980926514, test_accuracy = 0.6044999957084656
Epoch 6: training loss = 0.5946696996688843, training_time = 4.2045042514801025, val_accuracy = 0.60559